In [13]:
from functools import reduce
import numpy as np
import picos

class SageSDPProblem:
    def __init__(self, list_vars, equality_constraints, inequality_constraints,
                 psd_matrices, objective, debug=False):
        """
        Construct an Picos SDP problem from Sage variables.
        Args:
          list_vars: list of decision variables
          equality_constraints/inequality_constraints: list of linear functions f_i in 
            the decision variables `list_vars`. Used to impose the constraint f_i == 0 or f_i >= 0
          psd_matrices: list of matrices Q_i. Used to impose the constraint Q_i psd.
          objective: linear function in the decision variables used as objective function to maximize.
        """
        
        if debug:
            print('SDP with {} variables, involving {} matrices of sizes:'\
                  .format(len(list_vars), len(psd_matrices)))
            print(list(map(lambda M: M.dimensions()[0], psd_matrices)))
            
        self.prob = picos.Problem()
        self.list_vars = list_vars
        self.sage_equality_constraints = equality_constraints
        self.sage_inequality_constraints = inequality_constraints
        self.sage_psd_matrices = psd_matrices
        self.sage_objective = objective

        self._sage_to_pic = {}
        self._pic_equality_constraints = []
        self._pic_inequality_constraints = []
        self._pic_psd_constraints = []
        self._pic_objective = []

        # Sage -> Picos conversion
        self._build_picos_vars()
        self._build_equality_constraints()
        self._build_inequality_constraints()
        self._build_psd_matrices()
        self._build_objective()

    def _get_pic_var_from_sage_var(self, sage_var):
        return self._sage_to_pic.get(str(sage_var), None)

    
    def _get_pic_expr_from_sage_expr(self, sage_expr):
        sage_expr = SR(sage_expr)
        op = sage_expr.operands()
        # if sage_expr contains an operation
        if len(op) > 0:
            return sage_expr.operator()(*map(self._get_pic_expr_from_sage_expr, op))
        # otherwise, `sage_expr` is either a variable or a constant (i.e., a flot)
        else:
            pic_var = self._get_pic_var_from_sage_var(sage_expr)
            return pic_var if pic_var else float(sage_expr)
        
    def _build_picos_vars(self):
        for v in self.list_vars:
            if not(self._get_pic_var_from_sage_var(v)):
                self._sage_to_pic[str(v)] = picos.RealVariable(str(v))
            
    def _build_equality_constraints(self):
        pic_lhs = map(self._get_pic_expr_from_sage_expr, self.sage_equality_constraints)

        for eq in pic_lhs:
            constraint = self.prob.add_constraint(eq == float(0))
            self._pic_equality_constraints.append(constraint)
            
            
    def _build_inequality_constraints(self):
        pic_lhs = map(self._get_pic_expr_from_sage_expr, self.sage_inequality_constraints)

        for eq in pic_lhs:
            constraint = self.prob.add_constraint(eq >= float(0))
            self._pic_inequality_constraints.append(constraint)            
                     
                
    def _build_psd_matrices(self):
        for A in self.sage_psd_matrices:
            pic_A = [[self._get_pic_expr_from_sage_expr(Aij)
                      for Aij in Ai]
                     for Ai in A]
            if len(pic_A) == 1:
                self._pic_psd_constraints.append(pic_A[0][0] >= float(0))
                self.prob.add_constraint(pic_A[0][0] >= float(0))
                continue
            # build a picos matrix from pic_A
            pic_A = [reduce(lambda a, b: a & b, Ai) for Ai in pic_A]
            pic_A = reduce(lambda a, b: a // b, pic_A)

            self._pic_psd_constraints.append(pic_A >> float(0))
            self.prob.add_constraint(pic_A >> float(0))
        
    def _build_objective(self):
        self._pic_objective = self._get_pic_expr_from_sage_expr(self.sage_objective)
        self.prob.set_objective('max', self._pic_objective)
        
    def solve(self, **solver_params):
        """
        Solve SDP.
        Returns a tuple `(st, dict_solution)`, where `st` is the status returned by the solver, 
          and dict_solution is a dictionary mapping decision variables to their value.
        """
        solve_return = self.prob.solve(**solver_params)
        return solve_return.status,  {k.name: v[0] for k, v in solve_return.primals.items()}

    

def symb_matrix(n, name='Q'):
    """Construct an `n` x `n` symmetric symbolic matrix named `name`."""
    template_var_names = name+"_%d%d"
    order = lambda idx: (max(idx), min(idx))
    var_names = map(lambda idx: template_var_names % order(idx), 
                    cartesian_product([range(1, n+1),]*2))
    var_names = list(var_names)
    var_names = np.matrix(var_names).reshape(n, n)
    create_sym_entry = np.vectorize(lambda name: polygen(QQ, name))
    Q = create_sym_entry(var_names)
    Q = matrix(SR, Q)
    return Q


To search for the sos decomposition of a polynomial $p(x)$, we look for a psd matrix $Q$ such that

$$p(x) - m(x)^TQm(x) == 0 \quad \forall x,$$

where $m(x)$ is the vector of monomials in $x$. The identity above is equivalent to setting the coefficients of the polynomial $\Delta(x) := p(x) - m(x)^TQm(x)$ to zero.

The code below does exactly that.

In [18]:
# Polynomial ring in three variables
P.<x,y> = QQ['x, y']

p = 2*x^4+5*y^4-x^2*y^2+2*x^3*y
print("p(x,y) = ", p)

# vector of monomials
mons  = vector([x^2, y^2, x*y])
print("monomials = ", mons)

# psd matrix Q to search for
Q = symb_matrix(n=len(mons), name='Q')
print("Q = ", Q)

## setup sos constraint

# extract coefficients of diff := p - m'Qm
# and set them equal to 0
diff = QQ[Q.variables()][x,y](p - mons*Q*mons)
print("p - m^TQm = ", diff)

coefficients_diff = diff.coefficients()

sdp_problem = SageSDPProblem(
    list_vars=Q.variables(), # the decision variables are the entries of the matrix Q
    equality_constraints=coefficients_diff, # set the coefficients of p - m'Qm to 0
    inequality_constraints=[], # we do not need any inequality constraints here
    psd_matrices=[Q,], # we want the matrix Q to be psd
    objective=-Q.trace(), # maximize -tr(Q)
    debug=True)

# Solve using mosek
status, solution = sdp_problem.solve(solver='mosek', verbose=True)

# Print the solution
print("Optimal matrix Q found:")
print(Q.subs(**solution))

p(x,y) =  2*x^4 + 2*x^3*y - x^2*y^2 + 5*y^4
monomials =  (x^2, y^2, x*y)
Q =  [Q_11 Q_21 Q_31]
[Q_21 Q_22 Q_32]
[Q_31 Q_32 Q_33]
p - m^TQm =  (-Q_11 + 2)*x^4 + (-2*Q_31 + 2)*x^3*y + (-2*Q_21 - Q_33 - 1)*x^2*y^2 + (-2*Q_32)*x*y^3 + (-Q_22 + 5)*y^4
SDP with 6 variables, involving 1 matrices of sizes:
[3]
            PICOS 2.0.8            
Problem type: Semidefinite Program.
Searching a solution strategy for MOSEK (Optimizer).
Solution strategy:
  1. ExtraOptions
  2. MOSEKSolver
Applying ExtraOptions.
Building a MOSEK (Optimizer) problem instance.
Starting solution search.
-----------------------------------
      MOSEK via Optimizer API      
-----------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 11              
  Cones                  : 0               
  Scalar variables       : 6               
  Matrix variables     

/home/bachir/Dev/sagemath/SageMath/local/lib/python3.7/site-packages/sage/repl/ipython_kernel/__main__.py:106: DeprecationWarning: Option 'verbose' is deprecated, use 'verbosity'.
/home/bachir/Dev/sagemath/SageMath/local/lib/python3.7/site-packages/picos/solvers/solver_mosek.py:599: DeprecationWarning: Problem.verbosity is deprecated: Use Problem.options instead.
  self.int.putintparam(mosek.iparam.log, self.ext.verbosity())
/home/bachir/Dev/sagemath/SageMath/local/lib/python3.7/site-packages/picos/solvers/solver_mosek.py:728: DeprecationWarning: Problem.is_continuous is deprecated: Use continuous instead.
  if self.ext.is_continuous():
/home/bachir/Dev/sagemath/SageMath/local/lib/python3.7/site-packages/picos/solvers/solver_mosek.py:753: DeprecationWarning: Problem.is_continuous is deprecated: Use continuous instead.
  if self.ext.options.duals is not False and self.ext.is_continuous():
/home/bachir/Dev/sagemath/SageMath/local/lib/python3.7/site-packages/picos/solvers/solver_mosek.py: